In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Aug  7 15:24:34 2021
Fake job posting
@author: thha8003
"""

import os
import numpy as np
import pandas as pd

import seaborn as sns
''' Reading the data '''
path = r'G:\My Drive\Classroom\Fake_job_posting'
os.chdir(path)


db = pd.read_csv("fake_job_postings.csv", sep=',', encoding = 'Latin1')

db.shape
db.info()
db.isnull().sum()
missing_percentage=db.isnull().mean()*100
db.describe()
db.columns

db.head()
db_1 = db.drop(["job_id","title","location","department","salary_range","company_profile","benefits","function"], axis = 'columns')

db_1.columns
Imbalance = sns.countplot(db['fraudulent'])

'''###########################EDA############################################'''
db_new = db_1.dropna()
db_1.shape
db_new.shape
db_new.isnull().sum()
Imbalance = sns.countplot(db_new['fraudulent'])
'''############################hot endoing####################################'''
db_new.info()

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

 
#we have imported LabelEncoder and now we will use fit and transform method to covert sring to Numeric
'''############### Converting industry object column to numeric type##############'''
le_description =LabelEncoder()
db_new['description_num'] = le_description.fit_transform(db_new['description'])
db_new = db_new.drop(["description"], axis = 'columns')
db_new.info()

'''############### Converting requirements object column to numeric type##############'''
le_requirements =LabelEncoder()
db_new['requirements_num'] = le_requirements.fit_transform(db_new['requirements'])
db_new = db_new.drop(["requirements"], axis = 'columns')
db_new.info()

'''############### Converting employment_type object column to numeric type##############'''
le_employment_type = LabelEncoder()
db_new['employment_type_num'] = le_employment_type.fit_transform(db_new['employment_type'])
db_new.info()
db_new = db_new.drop(["employment_type"], axis = 'columns')

'''############### Converting required_experience object column to numeric type##############'''
le_required_experience =LabelEncoder()
db_new['required_experience_num'] = le_required_experience.fit_transform(db_new['required_experience'])
db_new.info()
db_new = db_new.drop(["required_experience"], axis = 'columns')

'''############### Converting required_education object column to numeric type##############'''
Le_required_education =LabelEncoder()
db_new['required_education_num'] = Le_required_education.fit_transform(db_new['required_education'])
db_new = db_new.drop(["required_education"], axis = 'columns')
db_new.info()

'''############### Converting industry object column to numeric type##############'''
le_industry =LabelEncoder()
db_new['industry_num'] = le_industry.fit_transform(db_new['industry'])
db_new = db_new.drop(["industry"], axis = 'columns')
db_new.info()


''' Finding correlation '''
correlation = db_new.corr()

'''Dropping employment_type_num as very low correlation score###############'''
db_new = db_new.drop(["employment_type_num"], axis ="columns")

correlation = db_new.corr()


sns.countplot( x='fraudulent',hue ='location', data = db)

sns.countplot( x='fraudulent',hue ='required_education', data = db)
sns.countplot( x='fraudulent',hue ='has_company_logo', data = db)

'''################# applying smote function to balance the data ###################### '''
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE

smote = SMOTE()


'''########################## Splitting test and train data ################################'''
train = db_new.drop(["fraudulent"],axis="columns")
test = db_new["fraudulent"]
train.shape
test.shape
X_smote, y_smote= smote.fit_resample(train,test)
X_smote.shape
y_smote.shape
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size = 0.2, random_state = 0)



'''###############################creating model################################'''

from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier()

rfm= RandomForestClassifier(n_estimators = 100, oob_score=True, max_depth=6)
rfm.fit(X_train,y_train)
rfm.score(X_train,y_train)##model score is 90.41%

rfm.fit(X_test,y_test)
rfm.score(X_test,y_test)#model score is 91.58%

'''############################### Hyper parameter tunnig #############################'''

rfm.feature_importances_


rfm = RandomForestClassifier()
parameters = {
  'n_estimators':np.arange(20,500,20),
  'max_depth':np.arange(1,10,2)
} 
    
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(rfm,parameters,cv=5)
cv.fit(train,test.values.ravel())
cv. best_params_ 
'''training the model based on HPT, {'max_depth': 9, 'n_estimators': 60 ########'''
rfm= RandomForestClassifier(n_estimators = 60, oob_score=True, max_depth=9)
rfm.fit(X_train,y_train)
rfm.score(X_train,y_train)##model score is 95.85%

rfm.fit(X_test,y_test)
rfm.score(X_test,y_test)#model score is 96.39%
